In [1]:
from line_variants import LINE_VARIANTS
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from transformers import AutoTokenizer, AutoModel

# Обучаем предсказание линии

In [3]:
# Загружаем токенизатор и модель
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")

# Подготовка данных для обучения
texts = []  # Список текстов
labels = []  # Список меток

# Заполните texts и labels используя ваш датасет STATION_VARIANTS
for station, variants in LINE_VARIANTS.items():
    for variant in variants:
        texts.append(variant)
        labels.append(station)

# Кодирование меток
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [4]:
def get_embeddings(texts):
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=32)
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state[:, 0, :].numpy()  # Берем эмбеддинги первого токена [CLS]
    return embeddings

X = get_embeddings(texts)

In [5]:
# Обучение классификатора KNN
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X, labels_encoded)

# Функция для предсказания станции
def predict_line(query):
    query_embedding = get_embeddings([query])
    prediction = clf.predict(query_embedding)
    return label_encoder.inverse_transform(prediction)[0]

In [6]:
# Тестирование
print(predict_line("Замскворецк"))

Замоскворецкая


# Сохраняем предсказывающую модель в файл

In [7]:
from joblib import dump

In [8]:
# Сохраняем классификатор KNN и LabelEncoder
dump(clf, 'lines_knn_classifier.joblib')
dump(label_encoder, 'lines_label_encoder.joblib')

# Сохраняем токенизатор
tokenizer.save_pretrained('lines_tokenizer')

# Сохраняем модель трансформера
model.save_pretrained('lines_rubert-tiny-model')